In [33]:
import pandas as pd
import numpy as np
import nltk
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, confusion_matrix

# Загрузка необходимых ресурсов NLTK
#nltk.download('punkt')

# Определение набора положительных и отрицательных слов
positive_words = ['великолепная', 'великолепный', 'супер', 'рекомендую', 'хорошо', 'понравилось', 'отлично', 'отличная', 'отличный', 'отличное', 'отличные', 'красиво', 
                  'красивая', 'красивый', 'красивое', 'красивые', 'закажу', 'нравится', 'идеально', 'идеальные', 
                  'восторг', 'волшебно', 'великолепно', 'приятный', 'удобно', 'хороший', 'классный', 'классная',
                  'классное', 'понравился']
negative_words = ['не супер', 'не рекомендую', 'отказ', 'плохо', 'маломерит', 'большемерит', 'не соответствует', 'брак', 'не понравился',
                  'не понравилось', 'не понравилась', 'сломан', 'ужас', 'отвратительно', 'отвратительное', 'отвратительные', 
                  'отвратительная', 'кошмар', 'кошмарный', 'кошмарное', 'порван', 'нитки торчат', 'мерзкий']

# Функция для предобработки текста
def preprocess_text(text):
    if pd.isna(text):
        return ''
    text = text.lower()  # Приведение к нижнему регистру
    text = re.sub(r'[^\w\s]', '', text)  # Удаление знаков препинания
    return text

# Функция для оценки настроения на основе списков слов
def get_custom_sentiment(text):
    if pd.isna(text):  # Обработка NaN
        return 'neutral'  # Или любое другое значение, которое вам нужно
    if not isinstance(text, str):
        return 'neutral'
    
    words = nltk.word_tokenize(text)
    num_positive = sum([1 for word in words if word in positive_words])
    num_negative = sum([1 for word in words if word in negative_words])
    if num_positive > num_negative:
        return 'positive'
    elif num_negative > num_positive:
        return 'negative'
    else:
        return 'neutral'

# Чтение данных из файла
tweets = pd.read_csv(r'C:\ANAC\GB_Ost\Data_sample\DZ9\27181_all_cards.csv', 
                     sep='\t',  
                     nrows=30000,  
                     on_bad_lines='skip')  

# Предобработка текста
tweets['text'] = tweets['text'].apply(preprocess_text)

# Применение функции get_custom_sentiment к столбцу 'text'
tweets['sentiment'] = tweets['text'].apply(get_custom_sentiment)

# Загрузка вручную размеченных данных
manual_labels = pd.read_csv(r"C:\ANAC\GB_Ost\Data_sample\DZ9\test.csv")
manual_labels['text'] = manual_labels['text'].apply(preprocess_text)

# Объединяем два DataFrame
combined_labels = pd.concat([manual_labels, tweets], ignore_index=True)

# Удаляем строки с пустыми значениями в столбце 'text' и 'sentiment'
combined_labels.dropna(subset=['text', 'sentiment'], inplace=True)

# Проверяем, есть ли пустые значения
print(combined_labels.isnull().sum())

X = combined_labels['text']  # Находим тексты
y = combined_labels['sentiment']  # Находим метки

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создание модели с векторизацией текста
model = make_pipeline(CountVectorizer(), MultinomialNB())


y_train = y_train.replace({1: 'positive', 0: 'neutral', -1: 'negative'})
y_test = y_test.replace({1: 'positive', 0: 'neutral', -1: 'negative'})

# Обучаем модель
model.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred = model.predict(X_test)

# Оценка модели
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


all_colors           0
annotation_id    30000
annotator        30000
brand               20
category             0
color                1
colors              90
cons             30100
created_at       30000
description         93
has_sizes            0
id               30000
isObscene            1
keyword              0
kinds              109
lead_time        30000
mark                 1
matchingSize        10
name                 0
pros             30100
reviewerName       100
sentiment            0
size                 1
text                 0
updated_at       30000
dtype: int64
[[  51  352   28]
 [   1 3021 1024]
 [   0  117 4436]]
              precision    recall  f1-score   support

    negative       0.98      0.12      0.21       431
     neutral       0.87      0.75      0.80      4046
    positive       0.81      0.97      0.88      4553

    accuracy                           0.83      9030
   macro avg       0.88      0.61      0.63      9030
weighted avg       0.84      0.

In [37]:
one = tweets['sentiment'].value_counts()
one

sentiment
positive    15150
neutral     13579
negative     1271
Name: count, dtype: int64